### 크롤링한 데이터 전처리

In [2]:
import pandas as pd
import numpy as np

kin = pd.read_csv("../dataset/edu/유치원 데이터 크롤링.csv")
kin.head()

,kin_type,kin_name,kin_addr
0,사립(사인),가람유치원,<span>서울특별시 강동구 상암로63길 39</span>
1,사립(법인),가명유치원,<span>서울특별시 중구 청파로 447-1</span>
2,사립(사인),강남유정유치원,<span>서울특별시 강남구 광평로34길 17</span>
3,사립(사인),강남유치원,<span>서울특별시 강남구 선릉로129길 31</span>
4,사립(사인),강남유치원,<span>서울특별시 동작구 상도로41가길 13</span>


In [3]:
# 오류 데이터 정제

kin['kin_addr'] = kin['kin_addr'].str[6:-7]
kin.head()

,kin_type,kin_name,kin_addr
0,사립(사인),가람유치원,서울특별시 강동구 상암로63길 39
1,사립(법인),가명유치원,서울특별시 중구 청파로 447-1
2,사립(사인),강남유정유치원,서울특별시 강남구 광평로34길 17
3,사립(사인),강남유치원,서울특별시 강남구 선릉로129길 31
4,사립(사인),강남유치원,서울특별시 동작구 상도로41가길 13


### 도로명 주소로 행정동 추출하기

In [5]:
# 도로명 주소로 행정동 변환 (카카오 api)

import requests

addrs = kin['kin_addr']
addrs

locations = []
for addr in addrs:
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(addr)
    headers = {
    ## 카카오 API의 REST API키를 아래 예시와 같이 입력해주세요
    ## "Authorization": "KakaoAK REST API키 입력 gogo"}
    "Authorization": "KakaoAK 8cd64c4299a983cac6fd3edef103f134"}
    place = requests.get(url, headers = headers).json()['documents']
    locations.append(place)

city = [] ## 구
town = [] ## 행정동
for i in range(len(locations)):
    
    try:
        city.append(locations[i][0].get('address').get('region_2depth_name'))
        town.append(locations[i][0].get('address').get('region_3depth_h_name'))
        
    except IndexError:
        print(i,'번째 주소 못가져옴', end ='')
        print()
        city.append('없음')
        town.append('없음')
    
    except AttributeError:
        city.append(locations[i][0].get('road_address').get('region_2depth_name'))
        town.append(locations[i][0].get('road_address').get('region_3depth_h_name'))

city_town = np.array([city,town]).T
df_temp = pd.DataFrame(city_town, columns = ['region_2depth_name','region_3depth_h_name'])
df_temp.head()

3 번째 주소 못가져옴
196 번째 주소 못가져옴


,region_2depth_name,region_3depth_h_name
0,강동구,명일2동
1,중구,중림동
2,강남구,수서동
3,없음,없음
4,동작구,상도1동


In [10]:
kin['addr_gu'] = df_temp['region_2depth_name']
kin['addr_dong'] = df_temp['region_3depth_h_name']
kin.head()

,kin_type,kin_name,kin_addr,addr_gu,addr_dong
0,사립(사인),가람유치원,서울특별시 강동구 상암로63길 39,강동구,명일2동
1,사립(법인),가명유치원,서울특별시 중구 청파로 447-1,중구,중림동
2,사립(사인),강남유정유치원,서울특별시 강남구 광평로34길 17,강남구,수서동
3,사립(사인),강남유치원,서울특별시 강남구 선릉로129길 31,없음,없음
4,사립(사인),강남유치원,서울특별시 동작구 상도로41가길 13,동작구,상도1동


In [11]:
# 데이터 타입 변환

kin['addr_gu'] = kin['addr_gu'].astype(str)
kin['addr_dong'] = kin['addr_dong'].astype(str)

# 변환 오류 데이터
kin[kin['addr_dong'] == '없음']

In [13]:
# 결측치 제거

kin = kin.drop(3)
kin = kin.drop(196)
kin.info()

# 유치원 형태 통일 (법인/사인, 단설/병설)
kin['kin_type'] = kin['kin_type'].str[:-4]
kin.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 756 entries, 0 to 757
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   kin_type   756 non-null    object
 1   kin_name   756 non-null    object
 2   kin_addr   756 non-null    object
 3   addr_gu    756 non-null    object
 4   addr_dong  756 non-null    object
dtypes: object(5)
memory usage: 35.4+ KB


### 구별 비율

In [15]:
kin_gu = kin.groupby(['addr_gu', 'kin_type'], as_index=False)['addr_dong'].count()
print(kin_gu)

   addr_gu kin_type  addr_dong
0      강남구       공립         17
1      강남구       사립         18
2      강동구       공립         11
3      강동구       사립         21
4      강북구       공립          7
5      강북구       사립         10
6      강서구       공립         18
7      강서구       사립         31
8      관악구       공립         16
9      관악구       사립         15
10     광진구       공립          8
11     광진구       사립         17
12     구로구       공립         14
13     구로구       사립         17
14     금천구       공립          7
15     금천구       사립          9
16     노원구       공립         20
17     노원구       사립         39
18     도봉구       공립         11
19     도봉구       사립         17
20    동대문구       공립          9
21    동대문구       사립         18
22     동작구       공립          8
23     동작구       사립         18
24     마포구       공립         13
25     마포구       사립         14
26    서대문구       공립          5
27    서대문구       사립         16
28     서초구       공립         10
29     서초구       사립         13
30     성동구       공립         14
31     성

In [22]:
# 비율 계산 함수

df = kin_gu

def calculate_school_type_ratio(dataframe):
    result = {}
    for _, row in dataframe.iterrows():
        구 = row['addr_gu']
        학교형태 = row['kin_type']
        학교수 = row['addr_dong']
        
        if 구 not in result:
            result[구] = {'공립': 0, '사립': 0}
        
        result[구][학교형태] += 학교수
    
    # 계산된 비율을 저장
    for 구 in result:
        total = result[구]['공립'] + result[구]['사립']
        if total > 0:
            result[구]['공립'] = result[구]['공립'] * 100 / total
            result[구]['사립'] = result[구]['사립'] * 100 / total
        else:
            result[구]['공립'] = 0
            result[구]['사립'] = 0
    
    return result

result = calculate_school_type_ratio(df)
kin_gu_ratio = pd.DataFrame(result)

kin_gu_ratio = kin_gu_ratio.T
kin_gu_ratio = kin_gu_ratio.reset_index()
kin_gu_ratio = kin_gu_ratio.rename(columns={'index': '구'})
kin_gu_ratio = kin_gu_ratio[['구', '공립']]
kin_gu_ratio.head()

,구,공립
0,강남구,48.571429
1,강동구,34.375000
2,강북구,41.176471
3,강서구,36.734694
4,관악구,51.612903


In [32]:
kin_gu_ratio['합계'] = kin_gu_ratio['addr_dong']
kin_gu_ratio

,구,공립,합계
0,강남구,48.571429,35
1,강동구,34.375000,32
2,강북구,41.176471,17
3,강서구,36.734694,49
4,관악구,51.612903,31
5,광진구,32.000000,25
6,구로구,45.161290,31
7,금천구,43.750000,16
8,노원구,33.898305,59
9,도봉구,39.285714,28


In [46]:
list = []

a = kin_gu_ratio['구']

for i in range(len(a)):
    list.append(len(kin[(kin['addr_gu'] == a[i]) & (kin['kin_type'] == '공립')]))

kin_gu_ratio['공립 수'] = list
kin_gu_ratio

[17,
 11,
 7,
 18,
 16,
 8,
 14,
 7,
 20,
 11,
 9,
 8,
 13,
 5,
 10,
 14,
 16,
 20,
 14,
 12,
 4,
 18,
 5,
 4,
 14]

In [76]:
# csv로 저장
kin_gu_ratio.rename(columns={'공립': '국공립 유치원 비율'}, inplace=True)
kin_fin = kin_gu_ratio[['구', '공립 수', '합계', '국공립 유치원 비율']]
kin_fin.to_csv("Final Data/교육_국공립유치원 비율_구별_2023.csv", index=False)

In [77]:
kin_fin

,구,공립 수,합계,국공립 유치원 비율
0,강남구,17,35,48.571429
1,강동구,11,32,34.375000
2,강북구,7,17,41.176471
3,강서구,18,49,36.734694
4,관악구,16,31,51.612903
5,광진구,8,25,32.000000
6,구로구,14,31,45.161290
7,금천구,7,16,43.750000
8,노원구,20,59,33.898305
9,도봉구,11,28,39.285714


### 동별 비율

In [69]:
kin_dong = kin.groupby(['addr_gu', 'addr_dong', 'kin_type'], as_index= False)['kin_name'].count()
kin_dong

,addr_gu,addr_dong,kin_type,kin_name
0,강남구,개포2동,공립,1
1,강남구,개포2동,사립,1
2,강남구,개포3동,사립,2
3,강남구,개포4동,공립,1
4,강남구,개포4동,사립,1
...,...,...,...,...
492,중랑구,신내2동,공립,2
493,중랑구,신내2동,사립,3
494,중랑구,중화1동,공립,1
495,중랑구,중화1동,사립,2


In [83]:
kin_12 = kin_dong.groupby(['addr_gu', 'addr_dong'], as_index= False)['kin_name'].sum()
kin_12.rename(columns={'kin_name': '합계'}, inplace=True)
kin_12

,addr_gu,addr_dong,합계
0,강남구,개포2동,2
1,강남구,개포3동,2
2,강남구,개포4동,2
3,강남구,논현1동,1
4,강남구,논현2동,2
...,...,...,...
355,중랑구,상봉1동,2
356,중랑구,신내1동,2
357,중랑구,신내2동,5
358,중랑구,중화1동,3


In [94]:
kin_temp = pd.merge(kin_12, ka[['addr_gu', 'addr_dong', 'kin_name']], on=['addr_gu', 'addr_dong'], how = 'outer')
kin_temp.rename(columns={'kin_name': '공립 수'}, inplace=True)
kin_temp['공립 수'] = kin_temp['공립 수'].fillna(0)
kin_temp['공립 수'] = kin_temp['공립 수'].astype(int)
kin_temp

,addr_gu,addr_dong,합계,공립 수
0,강남구,개포2동,2,1
1,강남구,개포3동,2,0
2,강남구,개포4동,2,1
3,강남구,논현1동,1,1
4,강남구,논현2동,2,1
...,...,...,...,...
355,중랑구,상봉1동,2,0
356,중랑구,신내1동,2,1
357,중랑구,신내2동,5,2
358,중랑구,중화1동,3,1


In [100]:
kin_temp.rename(columns={'addr_gu': '구', 'addr_dong': '동'}, inplace=True)
kin_temp['국공립 유치원 비율'] = kin_temp['공립 수']/kin_temp['합계'] * 100
kin_temp.to_csv('/Users/Jenny/Desktop/seoulbigcam/Final Data/교육_국공립유치원 비율_동별_2023.csv', index=False)

In [84]:
ka = kin_dong[kin_dong['kin_type'] == '공립']
ka

,addr_gu,addr_dong,kin_type,kin_name
0,강남구,개포2동,공립,1
3,강남구,개포4동,공립,1
5,강남구,논현1동,공립,1
6,강남구,논현2동,공립,1
9,강남구,대치2동,공립,1
...,...,...,...,...
485,중랑구,면목본동,공립,2
488,중랑구,묵2동,공립,2
490,중랑구,신내1동,공립,1
492,중랑구,신내2동,공립,2


In [62]:
# 비율 계산

df = kin_dong

def calculate_school_type_ratio(dataframe):
    result = {}
    for _, row in dataframe.iterrows():
        구 = row['addr_gu']
        동 = row['addr_dong']
        학교형태 = row['kin_type']
        학교수 = row['kin_name']
        
        if 동 not in result:
            result[동] = {'공립': 0, '사립': 0}
        
        result[동][학교형태] += 학교수
    
    # 계산된 비율을 저장
    for 동 in result:
        total = result[동]['공립'] + result[동]['사립']
        if total > 0:
            result[동]['공립'] = result[동]['공립'] * 100 / total
            result[동]['사립'] = result[동]['사립'] * 100 / total
        else:
            result[동]['공립'] = 0
            result[동]['사립'] = 0
    
    return result

result = calculate_school_type_ratio(df)
kin_dong_ratio = pd.DataFrame(result)

kin_dong_ratio = kin_dong_ratio.T
kin_dong_ratio = kin_dong_ratio.reset_index()
kin_dong_ratio = kin_dong_ratio.rename(columns={'index': '동'})
kin_dong_ratio = kin_dong_ratio[['동', '공립']]
kin_dong_ratio.head()

,동,공립
0,개포2동,50.0
1,개포3동,0.0
2,개포4동,50.0
3,논현1동,100.0
4,논현2동,50.0


In [49]:
# csv로 저장
kin_dong_ratio.to_csv("서울시 행정동별 국공립유치원 비율.csv")

In [1]:
import pandas as pd

kin_new = pd